<a href="https://colab.research.google.com/github/Krish5986/Project_Samarth_Q-A/blob/main/Data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import pandas as pd

# 🔑 Your API Credentials
API_KEY = "579b464db66ec23bdd00000175b6a0b8df884bc76684aecde95a1d86"
RESOURCE_ID = "35be999b-0208-4354-b557-f6ca9a5355de"
BASE_URL = f"https://api.data.gov.in/resource/{RESOURCE_ID}?api-key={API_KEY}&format=json"

# 🎯 Target Geographies and Time Period (5 States, 5 Years)
# NOTE: Replace 'BHOPAL', 'PUNE', etc., with 10 actual district names for each state
#       that are consistently spelled in the dataset (often in ALL CAPS).

TARGET_YEARS = [1997,1999,2000,2005, 2010,2011,2012]

TARGET_STATES_DISTRICTS = {
    "Uttar Pradesh": [
        "MEERUT", "AGRA", "LUCKNOW", "GHAZIABAD", "KANPUR NAGAR",
        "VARANASI", "ALIGARH", "MORADABAD", "GORAKHPUR", "ALLAHABAD" # PRAYAGRAJ -> ALLAHABAD
    ],
    "Madhya Pradesh": [
        "HOSHANGABAD", "VIDISHA", "SEHORE", "BHOPAL", "INDORE",
        "JABALPUR", "GWALIOR", "UJJAIN", "SAGAR", "RATLAM"
    ],
    "Maharashtra": [
        "KOLHAPUR", "AHMEDNAGAR", "NAGPUR", "NASHIK", "AURANGABAD",
        "SOLAPUR", "AMRAVATI", "THANE", "RAIGAD", "PUNE"
    ],
    "Karnataka": [
        "HASSAN", "CHICKMAGALUR", "MYSORE", "DAVANGERE", "SHIMOGA",
        "BELGAUM", "GULBARGA", "TUMKUR", "BIJAPUR", "BANGALORE RURAL"
        # CHICKMAGALUR is likely still going to fail, but we've maximized the rest.
    ],
    "Gujarat": [
        "KACHCHH", "SURENDRANAGAR", "SURAT", "VADODARA", "RAJKOT",
        "BHAVNAGAR", "JAMNAGAR", "GANDHINAGAR", "JUNAGADH", "ANAND"
    ]
}

# Now, re-run your previous data harvesting script using this updated dictionary.

# 🛠️ Pagination Settings
LIMIT = 500  # Number of records to fetch per API call
all_records = []

In [ ]:
print("--- Starting Data Harvest ---")
fetch_count = 0

for state, districts in TARGET_STATES_DISTRICTS.items():
    state_url_param = state.replace(' ', '%20') # URL encode spaces

    for district in districts:
        for year in TARGET_YEARS:

            # --- PHASE 1: Determine Total Records (Optional but good practice) ---
            # Run a query with limit=1 to find the total records available for this filter
            count_url = (
                f"{BASE_URL}&filters%5Bstate_name%5D={state_url_param}"
                f"&filters%5Bdistrict_name%5D={district}"
                f"&filters%5Bcrop_year%5D={year}&limit=1"
            )
            count_response = requests.get(count_url).json()
            TOTAL = int(count_response.get("total", 0))

            if TOTAL == 0:
                print(f"Skipping {district}, {state}, {year}: 0 records found.")
                continue

            # --- PHASE 2: Fetch All Records with Pagination ---
            offset = 0
            while offset < TOTAL:
                # 1. Construct the Full URL with all filters and pagination
                data_url = (
                    f"{BASE_URL}&filters%5Bstate_name%5D={state_url_param}"
                    f"&filters%5Bdistrict_name%5D={district}"
                    f"&filters%5Bcrop_year%5D={year}"
                    f"&limit={LIMIT}&offset={offset}"
                )

                # 2. Execute the request
                response = requests.get(data_url).json()

                # 3. Store records and update offset
                records = response.get("records", [])
                all_records.extend(records)

                offset += LIMIT
                fetch_count += 1
                print(f"  Fetched {len(records)} records for {district}, {year}. Total harvested: {len(all_records)}")

print(f"\n✅ Data Harvesting Complete! Total API calls: {fetch_count}")
print(f"Total Consolidated Records: {len(all_records)}")

# 4. Create the Consolidated Data Store (The foundation of your Q&A system)
consolidated_df = pd.DataFrame(all_records)
# Display size and sample
print("\n--- Consolidated DataFrame Snapshot ---")
print(consolidated_df.info())
print(consolidated_df.head())

# Save locally for the Q&A system
# consolidated_df.to_csv("crop_production_harvest.csv", index=False)
# consolidated_df.to_sql("crop_data", sqlite_conn, if_exists="replace") # Recommended for a DB

--- Starting Data Harvest ---
  Fetched 22 records for MEERUT, 1999. Total harvested: 22
  Fetched 22 records for MEERUT, 2000. Total harvested: 44
  Fetched 16 records for MEERUT, 2005. Total harvested: 60
  Fetched 22 records for MEERUT, 2010. Total harvested: 82
  Fetched 25 records for MEERUT, 2011. Total harvested: 107
  Fetched 27 records for MEERUT, 2012. Total harvested: 134
  Fetched 26 records for AGRA, 1999. Total harvested: 160
  Fetched 24 records for AGRA, 2000. Total harvested: 184
  Fetched 19 records for AGRA, 2005. Total harvested: 203
  Fetched 25 records for AGRA, 2010. Total harvested: 228
  Fetched 30 records for AGRA, 2011. Total harvested: 258
  Fetched 29 records for AGRA, 2012. Total harvested: 287
  Fetched 27 records for LUCKNOW, 1999. Total harvested: 314
  Fetched 23 records for LUCKNOW, 2000. Total harvested: 337
  Fetched 21 records for LUCKNOW, 2005. Total harvested: 358
  Fetched 28 records for LUCKNOW, 2010. Total harvested: 386
  Fetched 31 records f

In [ ]:
consolidated_df

,state_name,district_name,crop_year,season,crop,area_,production_
0,Uttar Pradesh,MEERUT,1999,Kharif,Arhar/Tur,656,481
1,Uttar Pradesh,MEERUT,1999,Kharif,Bajra,411,620
2,Uttar Pradesh,MEERUT,1999,Kharif,Castor seed,1,1
3,Uttar Pradesh,MEERUT,1999,Kharif,Cotton(lint),89,112
4,Uttar Pradesh,MEERUT,1999,Kharif,Jowar,40,31
...,...,...,...,...,...,...,...
8544,Gujarat,ANAND,2012,Summer,Rice,700,2200
8545,Gujarat,ANAND,2012,Whole Year,Banana,9800,775100
8546,Gujarat,ANAND,2012,Whole Year,Dry chillies,3800,4100
8547,Gujarat,ANAND,2012,Whole Year,Garlic,300,1800


In [ ]:
import requests
import pandas as pd

# 🔑 API Configuration
API_KEY = "579b464db66ec23bdd00000175b6a0b8df884bc76684aecde95a1d86"
RESOURCE_ID = "35be999b-0208-4354-b557-f6ca9a5355de"
BASE_URL = f"https://api.data.gov.in/resource/{RESOURCE_ID}?api-key={API_KEY}&format=json"
LIMIT = 500  # Pagination limit

# 🎯 Define Target States and Districts
TARGET_STATES_DISTRICTS = {
    "Uttar Pradesh": [
        "MEERUT", "AGRA", "LUCKNOW", "GHAZIABAD", "KANPUR NAGAR",
        "VARANASI", "ALIGARH", "MORADABAD", "GORAKHPUR", "ALLAHABAD"
    ],
    "Madhya Pradesh": [
        "HOSHANGABAD", "VIDISHA", "SEHORE", "BHOPAL", "INDORE",
        "JABALPUR", "GWALIOR", "UJJAIN", "SAGAR", "RATLAM"
    ],
    "Maharashtra": [
        "KOLHAPUR", "AHMEDNAGAR", "NAGPUR", "NASHIK", "AURANGABAD",
        "SOLAPUR", "AMRAVATI", "THANE", "RAIGAD", "PUNE"
    ],
    "Karnataka": [
        "HASSAN", "CHICKMAGALUR", "MYSORE", "DAVANGERE", "SHIMOGA",
        "BELGAUM", "GULBARGA", "TUMKUR", "BIJAPUR", "BANGALORE RURAL"
    ],
    "Gujarat": [
        "KACHCHH", "SURENDRANAGAR", "SURAT", "VADODARA", "RAJKOT",
        "BHAVNAGAR", "JAMNAGAR", "GANDHINAGAR", "JUNAGADH"
    ],
    "Punjab" :["LUDHIANA","PATIALA","AMRITSAR",
               "JALANDHAR","SANGRUR"],

    "West Bengal": ["BIRBHUM","BURDWAN","HOOGHLY","NADIA","MURSHIDABAD",
                    "DARJEELING"],

    "Rajasthan" :["BARMER",'JODHPUR',"BHARATPUR","JAIPUR","DHOLPUR",
                  "KOTA","BIKANER"],

    "Haryana" :["KARNAL","KURUKSHETRA","PANIPAT",
                "HISAR","YAMUNANAGAR","SIRSA"],
    "Andhra Pradesh" : ["EAST GODAVARI","WEST GODAVARI","KRISHNA",
                        "KURNOOL","ANANTAPUR","GUNTUR"]

}

# 🧠 Function to Harvest Data
def harvest_crop_data(start_year=1997, end_year=2014, save_path="crop_production_data.csv"):
    all_records = []
    fetch_count = 0
    TARGET_YEARS = list(range(start_year, end_year + 1))

    print(f"--- Starting Data Harvest for {start_year}-{end_year} ---")

    for state, districts in TARGET_STATES_DISTRICTS.items():
        state_url_param = state.replace(' ', '%20')  # encode spaces

        for district in districts:
            for year in TARGET_YEARS:

                count_url = (
                    f"{BASE_URL}&filters%5Bstate_name%5D={state_url_param}"
                    f"&filters%5Bdistrict_name%5D={district}"
                    f"&filters%5Bcrop_year%5D={year}&limit=1"
                )
                try:
                    count_response = requests.get(count_url, timeout=10).json()
                except Exception as e:
                    print(f"⚠️ Error fetching count for {district}, {state}, {year}: {e}")
                    continue

                TOTAL = int(count_response.get("total", 0))
                if TOTAL == 0:
                    print(f"Skipping {district}, {state}, {year}: 0 records found.")
                    continue

                offset = 0
                while offset < TOTAL:
                    data_url = (
                        f"{BASE_URL}&filters%5Bstate_name%5D={state_url_param}"
                        f"&filters%5Bdistrict_name%5D={district}"
                        f"&filters%5Bcrop_year%5D={year}"
                        f"&limit={LIMIT}&offset={offset}"
                    )
                    try:
                        response = requests.get(data_url, timeout=15).json()
                        records = response.get("records", [])
                        if records:
                            all_records.extend(records)
                            print(f"Fetched {len(records)} records for {district}, {year}. Total: {len(all_records)}")
                        offset += LIMIT
                        fetch_count += 1
                    except Exception as e:
                        print(f"⚠️ Error fetching data for {district}, {year}: {e}")
                        break

    print(f"\n✅ Data Harvest Complete! API calls: {fetch_count}")
    print(f"Total Records Harvested: {len(all_records)}")

    # 🧾 Convert to DataFrame and Save
    if all_records:
        consolidated_df = pd.DataFrame(all_records)
        consolidated_df.to_csv(save_path, index=False)
        print(f"📁 Data saved to: {save_path}")
        print(consolidated_df.info())
        print(consolidated_df.head())
        return consolidated_df
    else:
        print("❌ No records found. Nothing to save.")
        return pd.DataFrame()

# 🚀 Run the Function
df = harvest_crop_data(1997, 2014)


--- Starting Data Harvest for 1997-2014 ---
Fetched 24 records for MEERUT, 1997. Total: 24
Fetched 21 records for MEERUT, 1998. Total: 45
Fetched 22 records for MEERUT, 1999. Total: 67
Fetched 22 records for MEERUT, 2000. Total: 89
Fetched 22 records for MEERUT, 2001. Total: 111
Fetched 23 records for MEERUT, 2002. Total: 134
Fetched 22 records for MEERUT, 2003. Total: 156
Fetched 17 records for MEERUT, 2004. Total: 173
Fetched 16 records for MEERUT, 2005. Total: 189
Fetched 21 records for MEERUT, 2006. Total: 210
Fetched 25 records for MEERUT, 2007. Total: 235
Fetched 23 records for MEERUT, 2008. Total: 258
Fetched 25 records for MEERUT, 2009. Total: 283
Fetched 22 records for MEERUT, 2010. Total: 305
Fetched 25 records for MEERUT, 2011. Total: 330
Fetched 27 records for MEERUT, 2012. Total: 357
Fetched 27 records for MEERUT, 2013. Total: 384
Fetched 26 records for MEERUT, 2014. Total: 410
Fetched 26 records for AGRA, 1997. Total: 436
Fetched 28 records for AGRA, 1998. Total: 464
Fetc

In [ ]:
from google.colab import drive

# This command will prompt you to click a link, sign into your Google account,
# and grant permissions.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# ⚠️ Assuming your updated DataFrame is named 'df'
# You would need to ensure your updated data is loaded into a DataFrame variable first.
# For example, if you loaded your data, the variable holding it is 'df'.

# 1. Define the full path to your Google Drive folder
gdrive_path = '/content/drive/MyDrive/Build_For_Bharat/crop_production_data.csv'

# 2. Save the DataFrame to the specified Google Drive path
# The 'index=False' prevents pandas from writing the DataFrame row numbers as a column.
df.to_csv(gdrive_path, index=False)

print(f"✅ Successfully saved your data to: {gdrive_path}")

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Build_For_Bharat'

In [ ]:
# --- Define the location and file name ---
# The folder name you specified
folder_name = 'Build_For_Bharat'
# The desired name for the saved file
file_name = 'crop_production_data_updated.csv'

# Define the absolute path to the folder
gdrive_folder_path = f'/content/drive/MyDrive/{folder_name}'
# Define the absolute path to the file
gdrive_file_path = f'{gdrive_folder_path}/{file_name}'

# --- Create the folder if it doesn't exist ---
# 'exist_ok=True' prevents an error if the folder is already there.
try:
    os.makedirs(gdrive_folder_path, exist_ok=True)
    print(f"✅ Google Drive folder verified/created: {gdrive_folder_path}")
except Exception as e:
    print(f"❌ ERROR: Could not verify/create directory. Check mounting and folder name. Error: {e}")

❌ ERROR: Could not verify/create directory. Check mounting and folder name. Error: name 'os' is not defined


In [ ]:
# Assuming the DataFrame from your harvest function is stored in the variable 'df'
if not df.empty:
    df.to_csv(gdrive_file_path, index=False)
    print(f"🎉 Successfully saved {len(df)} records to: {gdrive_file_path}")
else:
    print("❌ Cannot save: The DataFrame 'df' is empty or not properly loaded.")

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Build_For_Bharat'

In [ ]:
source_path = '/content/crop_production_data.csv'
# OR if it's inside sample_data:
# source_path = '/content/sample_data/crop_production_data.csv'

In [ ]:
destination_path = '/content/drive/MyDrive/Build_For_Bharat/crop_production_data.csv'

In [ ]:
import shutil
import os

# --- ⚠️ Update this with the correct current location of your file ---
source_path = '/content/crop_production_data.csv'

# --- 📁 Update this to your desired folder in Google Drive ---
drive_folder_name = 'Build_For_Bharat'
destination_path = f'/content/drive/MyDrive/{drive_folder_name}/crop_production_data.csv'

# 1. Create the destination folder in Drive if it doesn't exist
os.makedirs(os.path.dirname(destination_path), exist_ok=True)

# 2. Copy the file
try:
    shutil.copyfile(source_path, destination_path)
    print(f"✅ Success! File copied to Google Drive at: {destination_path}")
except FileNotFoundError:
    print(f"❌ Error: The source file was not found at {source_path}")
    print("Please check the exact path of your file in the Colab 'Files' tab and update 'source_path'.")

✅ Success! File copied to Google Drive at: /content/drive/MyDrive/Build_For_Bharat/crop_production_data.csv
